# **Exploratory Data Analysis for Orders & Customers**

Sub-Text for heading: to be added after finishing this file.

## **EDA for Orders & Customers Summary**

- summary
- in
- points

## **Import Libraries**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## **Import Data**

In [2]:
orders = pd.read_csv('../data/processed/orders.csv')
order_items = pd.read_csv('../data/processed/order_items.csv')
customers = pd.read_csv('../data/processed/customers.csv')
payments = pd.read_csv('../data/processed/payments.csv')
products = pd.read_csv('../data/processed/products.csv')

## **Heading**